In [1]:
# import das bibliotecas
import pyspark
from pyspark.sql.types import *
from pyspark.sql.functions import *
from pyspark import SparkConf, SparkContext
from pyspark.sql import SparkSession, Row
import numpy as np
import pandas as pd
import collections
import os
from os.path import isfile, isdir, join

In [2]:
os.environ['PYSPARK_SUBMIT_ARGS'] = '--jars file:///home/jovyan/jdbc/postgresql-42.2.17.jar pyspark-shell'

In [3]:
db_host = os.getenv('POSTGRES_HOST')
db_port = os.getenv('POSTGRES_PORT')
db_name = os.getenv('POSTGRES_DB')
db_user = os.getenv('POSTGRES_USER')
db_pass = os.getenv('POSTGRES_PASSWORD')

db_driver = "org.postgresql.Driver"
db_url = "jdbc:postgresql://"+db_host+":"+db_port+"/" + db_name

In [4]:
# quando for True, as tabelas das dimensões serão recriadas e carregadas
carregar_dimensoes = True

In [5]:
# inicialização do spark
conf = SparkConf() \
        .setMaster("local[*]") \
        .setAppName("ETL-CarregarDadosNasDimensoes") \
        .set("spark.executor.memory", "8g") \
        .set("spark.driver.memory", "8g") \
        .set("spark.driver.maxResultSize", "2g") \
        .set("spark.ui.enabled", "true") \
        .set("spark.sql.shuffle.partitions" , "800") \
        .set("spark.sql.execution.arrow.pyspark.enabled" , "false") \

spark = SparkSession \
    .builder \
    .config(conf=conf) \
    .getOrCreate()

sc = spark.sparkContext

In [6]:
# definindo o schema dos dados para leitura dos arquivos JSON
schema = StructType([
    StructField("dadosBasicos", StructType([
        StructField("assunto", ArrayType(
            StructType([
                StructField("assuntoLocal", StructType([
                    StructField("codigoAssunto", LongType(), True),
                    StructField("codigoPaiNacional", LongType(), True),
                    StructField("descricao", StringType(), True)
                ]), True),
                StructField("codigoNacional", LongType(), True),
                StructField("principal", BooleanType(), True)
            ]),
        ), True),
        StructField('classeProcessual', LongType(), True),
        StructField('codigoLocalidade', StringType(), True),
        StructField('competencia', StringType(), True),
        StructField('dataAjuizamento', StringType(), True),
        StructField('dscSistema', StringType(), True),
        StructField('nivelSigilo', LongType(), True),
        StructField('numero', StringType(), True),
        StructField("orgaoJulgador", StructType([
            StructField("codigoMunicipioIBGE", LongType(), True),
            StructField("codigoOrgao", StringType(), True),
            StructField("instancia", StringType(), True),
            StructField("nomeOrgao", StringType(), True)
        ]), True),
        StructField('procEl', LongType(), True),
        StructField("tamanhoProcesso", StringType(), True),
        StructField("totalAssuntos", LongType(), True),
        StructField("valorCausa", StringType(), True)       
    ]), True),
    StructField("grau", StringType(), True),
    StructField("millisInsercao", LongType(), True),
    StructField("movimento", ArrayType(     
        StructType([
            StructField("complementoNacional", ArrayType(
                StructType([
                    StructField("codComplemento", LongType(), True),
                    StructField("codComplementoTabelado", LongType(), True),
                    StructField("descricaoComplemento", StringType(), True),
                ])
            ), True),
            StructField("dataHora", StringType(), True),
            StructField("idDocumentoVinculado", ArrayType(
                StringType(),
            ), True),
            StructField("identificadorMovimento", StringType(), True),
            StructField("movimentoLocal", StructType([
                StructField('codigoMovimento', LongType(), True),
                StructField('codigoPaiNacional', LongType(), True)
            ]), True),
            StructField("movimentoNacional", StructType([
                StructField('codigoNacional', LongType(), True)
            ]), True),
            StructField("nivelSigilo", StringType(), True),
            StructField("orgaoJulgador", StructType([
                StructField("codigoMunicipioIBGE", LongType(), True),
                StructField("codigoOrgao", StringType(), True),
                StructField("instancia", StringType(), True),
                StructField("nomeOrgao", StringType(), True)
            ]), True),
            StructField("tipoDecisao", StringType(), True),
            StructField("tipoResponsavelMovimento", StringType(), True)
        ]),
    ), True),
    StructField("siglaTribunal", StringType(), True)
])

In [7]:
# carrega o CSV de classes e faz a carga da dimensão
df_classes = spark.read \
    .option("header","true") \
    .option("inferSchema","true") \
    .option("delimiter",";") \
    .csv("./base/sgt_classes.csv")

df_classes.createOrReplaceTempView("classes")
   
df_qry_classes = spark.sql(
    "SELECT " +
    "codigo AS cod," + 
    "descricao," + 
    "sigla," + 
    "cod_pai AS codpai " +    
    "FROM classes "
)

if carregar_dimensoes :
    df_qry_classes.write \
        .mode("overwrite") \
        .format("jdbc") \
        .option("url", db_url).option("user", db_user).option("password", db_pass).option("driver", db_driver) \
        .option("dbtable", "inovacnj.classe") \
        .option("truncate", "true") \
        .save()

print("tabela inovacnj.classe criada.")

tabela inovacnj.classe criada.


In [8]:
# carrega o CSV de assuntos e faz a carga da dimensão
df_assuntos = spark.read \
    .option("header","true") \
    .option("inferSchema","true") \
    .option("delimiter",";") \
    .csv("./base/sgt_assuntos.csv")

df_assuntos.createOrReplaceTempView("assuntos")
   
df_qry_assuntos = spark.sql(
    "SELECT " +
    "codigo AS cod," + 
    "descricao," + 
    "cod_pai AS codpai " +    
    "FROM assuntos "
)

if carregar_dimensoes :
    df_qry_assuntos.write \
        .mode("overwrite") \
        .format("jdbc") \
        .option("url", db_url).option("user", db_user).option("password", db_pass).option("driver", db_driver) \
        .option("dbtable", "inovacnj.assunto") \
        .option("truncate", "true") \
        .save()

print("tabela inovacnj.assunto criada.")

tabela inovacnj.assunto criada.


In [9]:
# carrega o CSV de movimentos e faz a carga da dimensão
df_movimentos = spark.read \
    .option("header","true") \
    .option("inferSchema","true") \
    .option("delimiter",";") \
    .csv("./base/sgt_movimentos.csv")

# cria uma view temporaria dos movimentos
df_movimentos.createOrReplaceTempView("movimentos")

df_qry_movimentos = spark.sql(
    "SELECT " +
    "codigo AS cod," + 
    "descricao," + 
    "cod_pai AS codpai " +    
    "FROM movimentos "
)

if carregar_dimensoes :
    df_qry_movimentos.write \
        .mode("overwrite") \
        .format("jdbc") \
        .option("url", db_url).option("user", db_user).option("password", db_pass).option("driver", db_driver) \
        .option("dbtable", "inovacnj.movimentocnj") \
        .option("truncate", "true") \
        .save()

print("tabela inovacnj.movimentocnj criada.")

tabela inovacnj.movimentocnj criada.


In [10]:
# carrega o CSV de serventia e faz a carga da dimensão
df_serventias = spark.read \
    .option("header","true") \
    .option("inferSchema","true") \
    .option("delimiter",";") \
    .csv("./base/mpm_serventias.csv")

df_serventias.createOrReplaceTempView("serventias")

df_qry_serventias = spark.sql(
    "SELECT DISTINCT " +
    "cast(SEQ_ORGAO AS int) AS cod, " + 
    "NOMEDAVARA AS descricao, " + 
    "cast(INT_ORDEM_ORGAO as int) AS ordem, " + 
    "cast(SEQ_ORGAO_PAI AS int) AS codpai, " + 
    "TIP_ORGAO AS sigla_tipoj, " + 
    "DSC_TIP_ORGAO AS tipo_oj, " + 
    "DSC_CIDADE AS cidade, " + 
    "SIG_UF AS uf, " + 
    "cast(COD_IBGE AS int) AS codibge, " + 
    "TIP_ESFERA_JUSTICA AS esfera, " + 
    "cast(LATITUDE AS float) AS latitude, " + 
    "cast(LONGITUDE AS float) AS longitude " + 
    "FROM serventias " +
    "WHERE SEQ_ORGAO IS NOT NULL " 
)

df_oj_codtribunal = spark.read \
    .option("header","true") \
    .option("inferSchema","true") \
    .option("delimiter",",") \
    .csv("./base/orgaojulgador_codtribunal.csv")

df_oj_codtribunal = df_oj_codtribunal.select(
    col("codtribunal"), col("cod_orgao_julg")
).distinct()

df_oj_classificacao = spark.read \
    .option("header","true") \
    .option("inferSchema","true") \
    .option("delimiter",";") \
    .csv("./base/orgaojulgador_classificacao.csv")

df_oj_classificacao = df_oj_classificacao.select(
    col("cod_orgao_julg"), col("Atuacao_Vara").alias("atuacao_vara")
).distinct()

df_qry_serventias = df_qry_serventias \
    .join(df_oj_codtribunal, df_qry_serventias["cod"] == df_oj_codtribunal["cod_orgao_julg"], "left") \
    .join(df_oj_classificacao, df_qry_serventias["cod"] == df_oj_classificacao["cod_orgao_julg"], "left") \
    .select(
        col("cod"), col("descricao"), col("ordem"), col("codpai"), col("codtribunal"), 
        col("atuacao_vara"), col("sigla_tipoj"), col("tipo_oj"),
        col("cidade"), col("uf"), col("codibge"), col("esfera"),
        col("latitude"), col("longitude")
    )

if carregar_dimensoes :
    df_qry_serventias.write \
        .mode("overwrite") \
        .format("jdbc") \
        .option("url", db_url).option("user", db_user).option("password", db_pass).option("driver", db_driver) \
        .option("dbtable", "inovacnj.orgao_julgador") \
        .save()

    print("tabela inovacnj.orgao_julgador criada.")

In [11]:
# carrega o CSV de tribunal e faz a carga da dimensão
df_tribunais = spark.read \
    .option("header","true") \
    .option("inferSchema","true") \
    .option("delimiter",",") \
    .csv("./base/tribunal.csv")

df_tribunais.createOrReplaceTempView("tribunais")

df_qry_df_tribunais = spark.sql(
    "SELECT * " +
    "FROM tribunais "
)

if carregar_dimensoes :
    df_qry_df_tribunais.write \
        .mode("overwrite") \
        .format("jdbc") \
        .option("url", db_url).option("user", db_user).option("password", db_pass).option("driver", db_driver) \
        .option("dbtable", "inovacnj.tribunal") \
        .option("truncate", "true") \
        .save()

    print("tabela inovacnj.tribunal criada.")

In [12]:
# carrega o CSV de fases e faz a carga da dimensão
df_fases = spark.read \
    .option("header","true") \
    .option("inferSchema","true") \
    .option("delimiter",";") \
    .csv("./base/fases.csv")

df_fases.createOrReplaceTempView("fase")

df_qry_fases = spark.sql(
    "SELECT " +
    "cod_fase as cod, " + 
    "fase as descricao, " + 
    "cod_tribunal as cod_tribunal " + 
    "FROM fase "
)

if carregar_dimensoes :
    df_qry_fases.write \
        .mode("overwrite") \
        .format("jdbc") \
        .option("url", db_url).option("user", db_user).option("password", db_pass).option("driver", db_driver) \
        .option("dbtable", "inovacnj.fase") \
        .option("truncate", "true") \
        .option("cascadeTruncate", "true") \
        .save()

print("tabela inovacnj.fase criada.")

tabela inovacnj.fase criada.


In [13]:
# carrega o CSV de fases x movimentos e faz a carga da dimensão
df_fase_movimentos = spark.read \
    .option("header","true") \
    .option("inferSchema","true") \
    .option("delimiter",";") \
    .csv("./base/fases_movimentos.csv")

df_fase_movimentos.createOrReplaceTempView("fase_movimento")

df_qry_fase_movimentos = spark.sql(
    "SELECT " +
    "cod_movimento, " + 
    "cod_fase " + 
    "FROM fase_movimento "
)

if carregar_dimensoes :
    df_qry_fase_movimentos.write \
        .mode("overwrite") \
        .format("jdbc") \
        .option("url", db_url).option("user", db_user).option("password", db_pass).option("driver", db_driver) \
        .option("dbtable", "inovacnj.fase_movimento") \
        .option("truncate", "true") \
        .save()

print("tabela inovacnj.fase_movimento criada.")

tabela inovacnj.fase_movimento criada.


In [14]:
# carrega o CSV de grau_jurisdicao e faz a carga da dimensão
df_grau = spark.read \
    .option("header","true") \
    .option("inferSchema","true") \
    .option("delimiter",",") \
    .csv("./base/grau_jurisdicao.csv")

df_grau.createOrReplaceTempView("grau_jurisdicao")

df_qry_grau = spark.sql(
    "SELECT " +
    "grau AS cod, " + 
    "descricao, " + 
    "tipo " + 
    "FROM grau_jurisdicao "
)

if carregar_dimensoes :
    df_qry_grau.write \
        .mode("overwrite") \
        .format("jdbc") \
        .option("url", db_url).option("user", db_user).option("password", db_pass).option("driver", db_driver) \
        .option("dbtable", "inovacnj.grau_jurisdicao") \
        .option("truncate", "true") \
        .save()

print("tabela inovacnj.grau_jurisdicao criada.")

tabela inovacnj.grau_jurisdicao criada.


In [15]:
# carrega o CSV de naturezas e faz a carga da dimensão
df_naturezas = spark.read \
    .option("header","true") \
    .option("inferSchema","true") \
    .option("delimiter",";") \
    .csv("./base/naturezas.csv")

df_naturezas.createOrReplaceTempView("natureza")

df_qry_naturezas = spark.sql(
    "SELECT " +
    "cod, " + 
    "natureza as descricao " + 
    "FROM natureza "
)

if carregar_dimensoes :
    df_qry_naturezas.write \
        .mode("overwrite") \
        .format("jdbc") \
        .option("url", db_url).option("user", db_user).option("password", db_pass).option("driver", db_driver) \
        .option("dbtable", "inovacnj.natureza") \
        .option("truncate", "true") \
        .save()

print("tabela inovacnj.natureza criada.")

tabela inovacnj.natureza criada.


In [16]:
# carrega o CSV de naturezas x classes e faz a carga da dimensão
df_naturezas_classes = spark.read \
    .option("header","true") \
    .option("inferSchema","true") \
    .option("delimiter",";") \
    .csv("./base/naturezas_classes.csv")

df_naturezas_classes.createOrReplaceTempView("natureza_classe")

df_qry_naturezas_classes = spark.sql(
    "SELECT " +
    "cod_classe, " + 
    "cod_natureza " + 
    "FROM natureza_classe "
)

if carregar_dimensoes :
    df_qry_naturezas_classes.write \
        .mode("overwrite") \
        .format("jdbc") \
        .option("url", db_url).option("user", db_user).option("password", db_pass).option("driver", db_driver) \
        .option("dbtable", "inovacnj.natureza_classe") \
        .option("truncate", "true") \
        .save()

print("tabela inovacnj.natureza_classe criada.")

tabela inovacnj.natureza_classe criada.


In [ ]:
# carrega cluster com ID

df_cluster = spark.read \
.option("header","true") \
    .option("inferSchema","true") \
    .option("delimiter",";") \
    .csv("./base/descricao_cluster_com_id.csv")

df_cluster.createOrReplaceTempView("cluster")
df_cluster_qry = spark.sql("SELECT cast(id_cluster as int) as cod, " +
                     "Desc_Cluster  as descricao, " +
                      "Cluster_Novo  as nome " +
                    "FROM cluster "
                      )
if carregar_dimensoes :
    df_cluster_qry.write \
            .mode("overwrite") \
            .format("jdbc") \
            .option("url", db_url).option("user", db_user).option("password", db_pass).option("driver", db_driver) \
            .option("dbtable", "inovacnj.clusteroj") \
            .save()

print("tabela inovacnj.clusteroj criada.")

In [ ]:
# carrega cluster com ID e seus órgãos julgadores

df_cluster_oj = spark.read \
.option("header","true") \
    .option("inferSchema","true") \
    .option("delimiter",";") \
    .csv("./base/descricao_cluster_com_oj_id.csv")

df_cluster_oj.createOrReplaceTempView("cluster_oj")
df_cluster.createOrReplaceTempView("cluster")

df_query_join = spark.sql(
    "SELECT cluster.id_cluster as cod_cluster, cluster_oj.cod_orgao_julg "  +
    "FROM cluster_oj, cluster " +
    "WHERE cluster_oj.tipo_justica2 = cluster.tipo_justica " +
    "AND cluster_oj.Atuacao_Vara2 = cluster.Atuacao_Vara " +
    "AND cluster_oj.Cluster_Novo2 = cluster.Cluster_Novo "
)

if carregar_dimensoes :
    df_query_join.write \
        .mode("overwrite") \
        .format("jdbc") \
        .option("url", db_url).option("user", db_user).option("password", db_pass).option("driver", db_driver) \
        .option("dbtable", "inovacnj.clusteroj_orgjulg") \
        .save()

print("tabela inovacnj.clusteroj_orgjulg criada.")

In [17]:
df_qry_classes = df_qry_classes.withColumnRenamed("descricao", "descclasse")
df_qry_movimentos = df_qry_movimentos.withColumnRenamed("descricao", "descmovimento")

In [18]:
ramos = {'./base/justica_eleitoral': 'jele', './base/justica_estadual': 'jest', \
         './base/justica_federal': 'jfed', './base/justica_militar': 'jmil', \
         './base/justica_trabalho': 'jtra'}

In [14]:
# faz o carregamento de todos os arquivos em um único DataFrame,
# geracao do CSV com os dados consolidados
# cria a tabela fato com os movimentos processuais

basedir = "./base"

dirs_ramos_justica = [join(basedir, f) for f in os.listdir(basedir) if isdir(join(basedir, f))]

is_first = True

for dir_ramo_just in dirs_ramos_justica:
    print("Iniciando carregamento do ramo de justica: " + dir_ramo_just)
    sufixo_tabela = ramos.get(dir_ramo_just, 'default')
    print("Sufixo tabela para ramo de justica: " + sufixo_tabela)
    dirs_tribunais = [join(dir_ramo_just, f) for f in os.listdir(dir_ramo_just) if isdir(join(dir_ramo_just, f))]
    
    for dir_trib in dirs_tribunais:
        print("Iniciando carregamento do tribunal: " + dir_trib)
        
        arquivos = [join(dir_trib, f) for f in os.listdir(dir_trib) if isfile(join(dir_trib, f))]
        
        df_union_tribunal = spark.createDataFrame(spark.sparkContext.emptyRDD(), schema)
        
        for arq in arquivos:
            if arq.endswith(".DS_Store") :
                continue
                
            print("Carregando dataframe do arquivo: " + arq)
            df = spark.read.schema(schema).json(arq)
            df_union_tribunal = df_union_tribunal.union(df)
        
        # Cria uma view temporaria para o dataframe
        df_union_tribunal.createOrReplaceTempView("proc_movimentos")
        
        
        # Query para carregar os assuntos dos processos na dimensão: inovacnj.processo_assunto
        df_query_processo_assunto = spark.sql(
            "SELECT DISTINCT " + 
            "siglaTribunal AS codtribunal, " + 
            "dadosBasicos.numero AS npu, " + 
            "to_timestamp(dadosBasicos.dataAjuizamento, 'yyyyMMddHHmmss') AS dtajuizamento, "
            "dadosBasicos.classeProcessual AS codclasse, " +
            
            "dadosBasicos.orgaoJulgador.codigoOrgao AS oj_cod, " +
            "dadosBasicos.orgaoJulgador.instancia AS oj_instancia, " +
            "dadosBasicos.orgaoJulgador.nomeOrgao AS oj_descricao, " +
            
            "coalesce(exp_assunto.assunto.codigoNacional, exp_assunto.assunto.codigoNacional, -1) AS codassunto, " +
            "exp_assunto.assunto.principal AS assunto_principal, " + 
            "coalesce(exp_assunto.assunto.assuntoLocal.codigoAssunto, exp_assunto.assunto.assuntoLocal.codigoAssunto, -1) AS codassunto_local, " +
            "coalesce(exp_assunto.assunto.assuntoLocal.codigoPaiNacional, exp_assunto.assunto.assuntoLocal.codigoPaiNacional, -1) AS codassunto_pai, " +
            "exp_assunto.assunto.assuntoLocal.descricao AS descassunto_local " +
            
            "FROM proc_movimentos " + 
            "LATERAL VIEW explode(dadosBasicos.assunto) exp_assunto as assunto " +
            "WHERE cast(substring(dadosBasicos.dataAjuizamento,0,4) as INT) >= 2000 AND to_timestamp(dadosBasicos.dataAjuizamento, 'yyyyMMddHHmmss') >= to_timestamp('20000101000000', 'yyyyMMddHHmmss') "
        )
        
        df_query_processo_assunto = df_query_processo_assunto \
           .join(df_qry_classes, df_query_processo_assunto["codclasse"] == df_qry_classes["cod"], "left") \
           .join(df_qry_assuntos, df_query_processo_assunto["codassunto"] == df_qry_assuntos["cod"], "left") \
           .select( \
                col("codtribunal"), col("npu"), col("dtajuizamento"), \
                col("codclasse"), col("descclasse"), \
                col("oj_cod"), col("oj_instancia"), col("oj_descricao"), \
                col("codassunto"), col("descricao").alias("descassunto"), \
                col("assunto_principal"), col("codassunto_local"), col("descassunto_local"), col("codassunto_pai") \
        )
        
        
        # Query para formato em CSV
        df_query_processo_movimento = spark.sql(
            "SELECT DISTINCT " + 
            "siglaTribunal AS codtribunal, " + 
            "grau, " +
            "millisinsercao, " +

            "dadosBasicos.classeProcessual AS codclasse, " +
            "dadosBasicos.codigoLocalidade AS codlocalidade, " +
            "coalesce(dadosBasicos.competencia, dadosBasicos.competencia, -1) as competencia, " +
            "to_timestamp(dadosBasicos.dataAjuizamento, 'yyyyMMddHHmmss') AS dtajuizamento, "
            "dadosBasicos.dscSistema AS descsistema, " +
            "dadosBasicos.nivelSigilo AS nivelsigilo, " +
            "dadosBasicos.numero AS npu, " + 
            "dadosBasicos.orgaoJulgador.codigoMunicipioIBGE AS oj_codibge, " +
            "dadosBasicos.orgaoJulgador.codigoOrgao AS oj_cod, " +
            "dadosBasicos.orgaoJulgador.instancia AS oj_instancia, " +
            "dadosBasicos.orgaoJulgador.nomeOrgao AS oj_descricao, " +
            "dadosBasicos.procEl AS tramitacao, " +
            "dadosBasicos.tamanhoProcesso AS tamanhoprocesso, " +
            "dadosBasicos.valorCausa AS valorcausa, " +

            "exp_movimento.movimento.dataHora AS mov_dtmov, " +
            "exp_movimento.movimento.nivelSigilo AS mov_nivelsigilo, " +
            "exp_movimento.movimento.movimentoNacional.codigoNacional AS mov_cod, " +
            "exp_movimento.movimento.movimentoLocal.codigoMovimento AS mov_codlocal, " +
            "exp_movimento.movimento.movimentoLocal.codigoPaiNacional AS mov_codpainacional, " +

            "exp_movimento.movimento.orgaoJulgador.codigoMunicipioIBGE as mov_oj_codibge, " +
            "exp_movimento.movimento.orgaoJulgador.codigoOrgao as mov_oj_cod, " +
            "exp_movimento.movimento.orgaoJulgador.instancia as mov_oj_instancia, " +
            "exp_movimento.movimento.orgaoJulgador.nomeOrgao as mov_oj_descricao, " +

            "coalesce(exp_movimento.movimento.tipoDecisao, exp_movimento.movimento.tipoDecisao, -1) as mov_tpdecisao, " +
            "coalesce(exp_movimento.movimento.tipoResponsavelMovimento, exp_movimento.movimento.tipoResponsavelMovimento, -1) as mov_tprespmov " +

            "FROM proc_movimentos " + 
            "LATERAL VIEW explode(movimento) exp_movimento as movimento " + 
            "WHERE cast(substring(dadosBasicos.dataAjuizamento,0,4) as INT) >= 2000 AND to_timestamp(dadosBasicos.dataAjuizamento, 'yyyyMMddHHmmss') >= to_timestamp('20000101000000', 'yyyyMMddHHmmss') " + 
            "AND exp_movimento.movimento.movimentoNacional.codigoNacional NOT IN(581, 85, 12270, 12271) " + 
            "AND size(proc_movimentos.movimento) > 0 "
            #"AND (proc_movimentos.movimento[0].movimentoNacional.codigoNacional IN (26, 12474) " +
            #"AND proc_movimentos.movimento[size(proc_movimentos.movimento) -1].movimentoNacional.codigoNacional IN (22, 246)) "
        )
        
        df_query_processo_movimento = df_query_processo_movimento \
           .join(df_qry_movimentos, df_query_processo_movimento["mov_cod"] == df_qry_movimentos["cod"], "left") \
           .join(df_qry_classes, df_query_processo_movimento["codclasse"] == df_qry_classes["cod"], "left") \
           .select( \
                col("codtribunal"), col("grau"), col("millisinsercao"), col("codclasse"), col("descclasse"), \
                col("codlocalidade"), col("competencia"), col("dtajuizamento"), col("descsistema"), \
                col("nivelsigilo"), col("npu"), col("valorcausa"), col("tramitacao"), col("tamanhoprocesso"), \
                col("oj_codibge"), col("oj_cod"), col("oj_instancia"), col("oj_descricao"), \
                col("mov_dtmov"), col("mov_cod"), col("descmovimento"), col("mov_codlocal"), \
                col("mov_codpainacional"), col("mov_nivelsigilo"), col("mov_oj_codibge"), \
                col("mov_oj_cod"), col("mov_oj_instancia"), col("mov_oj_descricao"), col("mov_tpdecisao"), \
                col("mov_tprespmov") \
        )
        
        #df_query_distinctPd = df_query_processo_movimento.toPandas()
        #df_query_distinctPd.to_csv('./output/movimentos_tribunais.csv', mode='a', header=is_first, sep = ";", index=False, chunksize=1000)
        
        df_query_processo_movimento = df_query_processo_movimento.withColumn('mov_dtmov', to_timestamp(df_query_processo_movimento['mov_dtmov'], 'yyyyMMddHHmmss'))
        
        if is_first == True:
            is_first = False
            
            df_query_processo_assunto.repartition(5).write \
                .mode("overwrite") \
                .format("jdbc") \
                .option("url", db_url).option("user", db_user).option("password", db_pass).option("driver", db_driver) \
                .option("dbtable", "inovacnj.processo_assunto") \
                .option("batchsize", "10000") \
                .save()
            
            df_query_processo_movimento.repartition(5).write \
                .mode("overwrite") \
                .format("jdbc") \
                .option("url", db_url).option("user", db_user).option("password", db_pass).option("driver", db_driver) \
                .option("dbtable", "inovacnj.fat_movimento_" + sufixo_tabela) \
                .option("batchsize", "10000") \
                .save()
        else :
            
            df_query_processo_assunto.repartition(5).write \
                .mode("append") \
                .format("jdbc") \
                .option("url", db_url).option("user", db_user).option("password", db_pass).option("driver", db_driver) \
                .option("dbtable", "inovacnj.processo_assunto") \
                .option("batchsize", "10000") \
                .save()
            
            df_query_processo_movimento.repartition(5).write \
                .mode("append") \
                .format("jdbc") \
                .option("url", db_url).option("user", db_user).option("password", db_pass).option("driver", db_driver) \
                .option("dbtable", "inovacnj.fat_movimento_" + sufixo_tabela) \
                .option("batchsize", "10000") \
                .save()

        print("Finalizando carregamento do tribunal: " + dir_trib)
        
    print("Finalizando carregamento do ramo de justica: " + dir_ramo_just)
    
print("Carregamento dos arquivos finalizado.")


Iniciando carregamento do ramo de justica: ./base/justica_militar
Sufixo tabela para ramo de justica: jmil
Iniciando carregamento do tribunal: ./base/justica_militar/processos-tjmsp
Carregando dataframe do arquivo: ./base/justica_militar/processos-tjmsp/processos-tjmsp_2.json
Finalizando carregamento do tribunal: ./base/justica_militar/processos-tjmsp
Finalizando carregamento do ramo de justica: ./base/justica_militar
Iniciando carregamento do ramo de justica: ./base/justica_trabalho
Sufixo tabela para ramo de justica: jtra
Iniciando carregamento do tribunal: ./base/justica_trabalho/processos-trt6
Carregando dataframe do arquivo: ./base/justica_trabalho/processos-trt6/processos-trt6_1.json
Finalizando carregamento do tribunal: ./base/justica_trabalho/processos-trt6
Finalizando carregamento do ramo de justica: ./base/justica_trabalho
Iniciando carregamento do ramo de justica: ./base/justica_eleitoral
Sufixo tabela para ramo de justica: jele
Iniciando carregamento do tribunal: ./base/jus

Finalizando carregamento do tribunal: ./base/justica_federal/processos-trf5
Finalizando carregamento do ramo de justica: ./base/justica_federal
Carregamento dos arquivos finalizado.


In [ ]:
# carrega o CSV de duracao_total_sf e faz a carga da dimensão
df_duracao_sf = spark.read \
    .option("header","true") \
    .option("inferSchema","true") \
    .option("delimiter",";") \
    .csv("./base/duracao_total_sf.csv")

df_duracao_sf.createOrReplaceTempView("duracao_total_sf")

df_qry_duracao_sf = spark.sql(
    "SELECT " +
    "codtribunal as codtribunal, " + 
    "grau as grau, " + 
    "cod_orgao_julg as cod_orgjulg, " + 
    "descricao as desc_orgjulg, " + 
    "tipo_justica as tipo_justica, " + 
    "uf as siglauf, " + 
    "tipo_processo as natureza, " + 
    "Porte_Municipio as porte_municipio, " + 
    "Porte_Tribunal as porte_tribunal, " + 
    "PIB_percapita as pib_percapita, " + 
    "Atuacao_Vara as atuacao_vara, " + 
    "Classificacao_Acervo_2 as classificacao_acervo, " + 
    "mediana as mediana, " + 
    "qtd_npu as qtd_npu, " + 
    "num_dias as num_dias " + 
    "FROM duracao_total_sf "
)

if carregar_dimensoes :
    df_qry_duracao_sf.write \
        .mode("overwrite") \
        .format("jdbc") \
        .option("url", db_url).option("user", db_user).option("password", db_pass).option("driver", db_driver) \
        .option("dbtable", "inovacnj.duracao_total_sf") \
        .option("truncate", "true") \
        .save()

print("tabela inovacnj.duracao_total_sf criada.")

In [ ]:
# carrega o CSV de duracao_total_cf e faz a carga da dimensão
df_duracao_cf = spark.read \
    .option("header","true") \
    .option("inferSchema","true") \
    .option("delimiter",";") \
    .csv("./base/duracao_total_cf.csv")

df_duracao_cf.createOrReplaceTempView("duracao_total_cf")

df_qry_duracao_cf = spark.sql(
    "SELECT " +
    "codtribunal as codtribunal, " + 
    "grau as grau, " + 
    "cod_orgao_julg as cod_orgjulg, " + 
    "descricao as desc_orgjulg, " + 
    "tipo_justica as tipo_justica, " + 
    "uf as siglauf, " + 
    "tipo_processo as natureza, " + 
    "Porte_Municipio as porte_municipio, " + 
    "Porte_Tribunal as porte_tribunal, " + 
    "PIB_percapita as pib_percapita, " + 
    "Atuacao_Vara as atuacao_vara, " + 
    "Classificacao_Acervo_2 as classificacao_acervo, " + 
    "mediana as mediana, " + 
    "qtd_npu as qtd_npu, " + 
    "num_dias as num_dias " + 
    "FROM duracao_total_cf "
)

if carregar_dimensoes :
    df_qry_duracao_cf.write \
        .mode("overwrite") \
        .format("jdbc") \
        .option("url", db_url).option("user", db_user).option("password", db_pass).option("driver", db_driver) \
        .option("dbtable", "inovacnj.duracao_total_cf") \
        .option("truncate", "true") \
        .save()

print("tabela inovacnj.duracao_total_cf criada.")

In [ ]:
# carrega o CSV de tempo e faz a carga da dimensão

df_tempo = spark.read \
.option("header","true") \
    .option("inferSchema","true") \
    .option("delimiter",",") \
    .csv("./base/tempo.csv")

df_tempo.createOrReplaceTempView("tempo")
df_tempo = spark.sql("SELECT cast(data as date) as data, " +
                     "nu_ano,  " +
                     "nu_dia,  " +
                     "nu_semestre, " +
                     "ds_semestre, " +
                     "ds_codanosemestre , " +
                     "ds_semestreano, " +
                     "nu_trimestre, " +
                     "ds_trimestre , " +
                     "nu_codanotrimestre , " +
                     "ds_trimestreano , " +
                     "nu_mes , " +
                     "ds_mes , " +
                     "sigla_mes , " +
                     "nu_anomes , " +
                     "ds_anomes , " +
                     "nu_diasemana , " +
                     "ds_diasemana , " +
                     "ds_siglasemana " +
                    "FROM tempo "
                      )
if carregar_dimensoes :
    df_tempo.write \
            .mode("overwrite") \
            .format("jdbc") \
            .option("url", db_url).option("user", db_user).option("password", db_pass).option("driver", db_driver) \
            .option("dbtable", "inovacnj.tempo") \
            .option("truncate", "true") \
            .save()

print("tabela inovacnj.tempo criada.")

In [ ]:
# carrega o CSV pib_municipio e faz a carga da dimensão

df_tempo = spark.read \
.option("header","true") \
    .option("inferSchema","true") \
    .option("delimiter",",") \
    .csv("./base/pib_municipio.csv")

df_tempo.createOrReplaceTempView("pibmunicipio")
df_tempo = spark.sql("SELECT cast(codmunicipio as int) as codmunicipio, " +
                         "porte,  " +
                         "cast(pib as float) as pib,  " +
                         "cast(pib_percapita as float) as pib_percapita, " +
                         "cast(pop_estimada as int) as pop_estimada, " +
                         "reg_metropolitana , " +
                         "desc_pibpercapita " +
                       "FROM pibmunicipio "
                      )
if carregar_dimensoes :
    df_tempo.write \
            .mode("overwrite") \
            .format("jdbc") \
            .option("url", db_url).option("user", db_user).option("password", db_pass).option("driver", db_driver) \
            .option("dbtable", "inovacnj.pib_municipio") \
            .option("truncate", "true") \
            .save()

print("tabela inovacnj.pib_municipio1 criada.")

In [12]:
import psycopg2
import os

import pandas.io.sql as sqlio

import pm4py
from pm4py.objects.log.util import dataframe_utils
from pm4py.objects.conversion.log import converter as log_converter
from pm4py.objects.conversion.dfg import converter as dfg_mining
from pm4py.algo.discovery.dfg import algorithm as dfg_discovery
from pm4py.algo.discovery.alpha import algorithm as alpha_miner
from pm4py.algo.filtering.log.variants import variants_filter
from pm4py.algo.conformance.tokenreplay import algorithm as token_replay
from pm4py.visualization.dfg import visualizer as dfg_visualization
from pm4py.visualization.petrinet import visualizer as pn_visualizer
from pm4py.statistics.traces.log import case_statistics
from pm4py.statistics.traces.log import case_arrival
from pm4py.statistics.start_activities.log import get as start_activities
from pm4py.statistics.end_activities.log import get as end_activities
from pm4py.objects.stochastic_petri import ctmc

import timer3

In [6]:
# Cache para logs de evento
eventLogCache = {}

# Remove um log de eventos da cache pela chave
def clear_eventlog_cache(cacheKey):
    eventLog = eventLogCache.get(cacheKey)

    if eventLog is not None :
        eventLogCache[cacheKey] = None

In [7]:
# Mapa chave=valor - ramo de justica/sufixo_tabela_fato
ramos_justica = {'Eleitoral': 'jele', 'Estadual': 'jest', 'Federal': 'jfed', 'Militar': 'jmil', 'Trabalho': 'jtra'}

In [18]:
# gera um log de eventos de acordo com os parametros informados.
def gerar_log_eventos(ramo_justica, codtribunal, atuacao, grau, codorgaoj, codnatureza, codclasse, dtinicio, dtfim, 
                      baixado = None, sensibility = '60'):
    
    eventLog = None

    cacheKey = "{0}-{1}-{2}-{3}-{4}-{5}-{6}-{7}-{8}-{9}".format(ramo_justica, codtribunal, atuacao, grau, codorgaoj, codnatureza, codclasse, dtinicio, dtfim, baixado)
    
    cachedEventLog = None #eventLogCache.get(cacheKey)
    if cachedEventLog is not None :
        eventLog = cachedEventLog

    else :
        conn = psycopg2.connect(host=db_host, port=db_port, database=db_name, user=db_user, password=db_pass)
        
        sufixo_ramo = ramos_justica.get(ramo_justica, 'default')
        
        tabela_fato = "inovacnj.fat_movimento_" + sufixo_ramo
        
        qry = "SELECT "
        qry+= "  fat.npu as npu, "
        qry+= "  CASE "
        qry+= "  WHEN f.descricao IS NULL THEN fat.mov_cod ||  ' - ' || mov.descricao "
        qry+= "  ELSE f.descricao || ': ' || fat.mov_cod ||  ' - ' || mov.descricao "
        qry+= "  END AS atividade, "
        qry+= "  fat.mov_dtmov as mov_dtmov "
        qry+= "FROM " + tabela_fato + " fat "
        qry+= "INNER JOIN inovacnj.acervo_processo_" + sufixo_ramo + " ap ON ap.npu = fat.npu "
        
        if codorgaoj is not None or atuacao is not None :
            qry+= "INNER JOIN inovacnj.orgao_julgador oj ON oj.cod::varchar = fat.oj_cod "
        
        qry+= "INNER JOIN inovacnj.movimentocnj mov ON mov.cod = fat.mov_cod "
        qry+= "INNER JOIN inovacnj.natureza_classe nc ON nc.cod_classe = fat.codclasse "
        qry+= "INNER JOIN inovacnj.natureza nat ON nat.cod = nc.cod_natureza "
        qry+= "LEFT JOIN inovacnj.fase_movimento fm ON fm.cod_movimento = fat.mov_cod "
        qry+= "LEFT JOIN inovacnj.fase f ON f.cod = fm.cod_fase "
        qry+= "WHERE (1=1) "
        
        if baixado is not None :
            qry+= "AND ap.baixado = '" + baixado + "' "
        if codtribunal is not None :
            qry+= "AND fat.codtribunal = '" + codtribunal + "' "
        if codorgaoj is not None :
            qry+= "AND fat.oj_cod = '" + codorgaoj + "' "
        if atuacao is not None :
            qry+= "AND oj.atuacao_vara = '" + atuacao + "' "
        if grau is not None :
            qry+= "AND fat.grau = '" + grau + "' "
        if codnatureza is not None :
            qry+= "AND nat.cod = " + str(codnatureza) + " "
        if codclasse is not None :
            qry+= "AND fat.codclasse = " + str(codclasse) + " "
            
        if dtinicio is not None and dtfim is not None:
            qry+= "AND fat.mov_dtmov BETWEEN to_timestamp('" + dtinicio + "', 'yyyy-MM-dd') AND to_timestamp('" + dtfim + "', 'yyyy-MM-dd') "
            
        qry+= "ORDER BY fat.npu, fat.mov_dtmov ASC "
        
        df_logeventos_pd = pd.read_sql_query(qry, conn)
        
        if df_logeventos_pd.empty == False :
            df_event_log = pm4py.format_dataframe(df_logeventos_pd, case_id='npu', activity_key='atividade', timestamp_key='mov_dtmov')
            eventLog = pm4py.convert_to_event_log(df_event_log)

            eventLogCache[cacheKey] = eventLog
            timer3.apply_after(1000 * 60 * 15, clear_eventlog_cache, args=([cacheKey]), priority=0)

    if eventLog is not None :
        if sensibility is not None :
            eventLog = pm4py.filter_variants_percentage(eventLog, percentage=float(sensibility) / 100)

    return eventLog

In [25]:
# executa um token replay em um log de eventos
def get_token_replayed_traces_from_params(net, initial_marking, final_marking,
                             ramo_justica, codtribunal, atuacao, grau, codorgaoj, codnatureza, codclasse, 
                             dtinicio, dtfim, baixado = None, sensibility = '60'):
    eventLog_oj = gerar_log_eventos(ramo_justica, codtribunal, atuacao, grau, codorgaoj, codnatureza, codclasse, dtinicio, dtfim, baixado, sensibility)
    if eventLog_oj is not None :
        return token_replay.apply(eventLog_oj, net, initial_marking, final_marking)
   
    return None

In [10]:
schemaModelFitness = StructType([
    StructField("tipo", StringType(), True),
    StructField("codtribunal", StringType(), True),
    StructField("atuacao_vara", StringType(), True),
    StructField("cod_orgao_julg", StringType(), True),
    StructField("desc_orgao_julg", StringType(), True),
    StructField("trace_fitness", FloatType(), True)
])

In [27]:
df_tribunal_atuacao_vara = spark.read.format("jdbc") \
    .option("url", db_url).option("user", db_user).option("password", db_pass).option("driver", db_driver) \
    .option("query", "select tr.tipo, codtribunal, atuacao_vara " +
        "from inovacnj.orgao_julgador oj " + 
        "inner join inovacnj.tribunal tr ON tr.cod = oj.codtribunal " + 
        "WHERE atuacao_vara is not NULL " +
        "AND tr.tipo NOT IN ('Eleitoral', 'Estadual') " +
#        "AND codtribunal NOT IN ('TJAC', 'TJAL', 'TJAM', 'TJAP', 'TJBA', 'TJCE', " +
#        "'TJDFT', 'TJES', 'TJGO', 'TJMA', 'TJPE', 'TJMT', 'TJPA', 'TJPB', 'TJPI', " +
#        "'TJPR', 'TJRJ', 'TJRN', 'TJRO', 'TJRR', 'TJRS', 'TJSC') "+
#        "AND (codtribunal <> 'TJMG' AND atuacao_vara <> 'Criminal') " +
#        "AND codtribunal = 'TJPE' " +
        "GROUP BY tr.tipo, codtribunal, atuacao_vara " +
        "ORDER BY tr.tipo, codtribunal, atuacao_vara") \
    .option("inferSchema","true") \
    .load()


is_first = True
carregar_dimensoes = True


for rowTribunalAtuacao in df_tribunal_atuacao_vara.rdd.collect():
    
    print(rowTribunalAtuacao['tipo'], rowTribunalAtuacao['codtribunal'], rowTribunalAtuacao['atuacao_vara'])
    
    eventLogTribunalAtuacao = gerar_log_eventos(rowTribunalAtuacao['tipo'], rowTribunalAtuacao['codtribunal'], 
                                                rowTribunalAtuacao['atuacao_vara'], None, None, None, None, 
                                                None, None, 'S', '1')
    
    net, initial_marking, final_marking = alpha_miner.apply(eventLogTribunalAtuacao)
    
    df_tribunal_atuacao_vara_oj = spark.read.format("jdbc") \
        .option("url", db_url).option("user", db_user).option("password", db_pass).option("driver", db_driver) \
        .option("query", "select cod, descricao " +
            "from inovacnj.orgao_julgador " + 
            "where atuacao_vara is not NULL " +
            "AND codtribunal = '"+ rowTribunalAtuacao['codtribunal'] +"' " +
            "AND atuacao_vara = '"+ rowTribunalAtuacao['atuacao_vara'] +"' " +
            "group by cod, descricao " +
            "order by cod, descricao") \
        .option("inferSchema","true") \
        .load()
    
    for rowTribunalAtuacaoVara in df_tribunal_atuacao_vara_oj.rdd.collect():
        
        print(rowTribunalAtuacao['tipo'], rowTribunalAtuacao['codtribunal'], rowTribunalAtuacao['atuacao_vara'], rowTribunalAtuacaoVara['descricao'])
        
        replayed_traces = get_token_replayed_traces_from_params(net, initial_marking, final_marking,
            rowTribunalAtuacao['tipo'], rowTribunalAtuacao['codtribunal'], rowTribunalAtuacao['atuacao_vara'], 
            None, str(rowTribunalAtuacaoVara['cod']), None, None, None, None, 'S', '1')
        
        if replayed_traces is not None :
            item = {
                "tipo": rowTribunalAtuacao['tipo'],
                "codtribunal": rowTribunalAtuacao['codtribunal'],
                "atuacao_vara": rowTribunalAtuacao['atuacao_vara'],
                "cod_orgao_julg": rowTribunalAtuacaoVara['cod'],
                "desc_orgao_julg": rowTribunalAtuacaoVara['descricao'],
                "trace_fitness": float(replayed_traces[0]['trace_fitness'])
            }

            df_orgao_julgador_fitness = spark.createDataFrame([item], schemaModelFitness)

            if carregar_dimensoes :
                df_orgao_julgador_fitness.write \
                    .mode("append") \
                    .format("jdbc") \
                    .option("url", db_url).option("user", db_user).option("password", db_pass).option("driver", db_driver) \
                    .option("dbtable", "inovacnj.fitnessmodel_org_julg_atuacao") \
                    .save()

                if is_first == True:
                    is_first = False

if carregar_dimensoes :
    print("tabela inovacnj.fitnessmodel_org_julg_atuacao criada.")

Federal TRF2 Criminal
Federal TRF2 Criminal 2ª VARA FEDERAL CRIMINAL DO RIO DE JANEIRO
Federal TRF2 Criminal 9ª VARA FEDERAL CRIMINAL DO RIO DE JANEIRO
Federal TRF2 Criminal 1ª VARA FEDERAL CRIMINAL DO RIO DE JANEIRO
Federal TRF2 Criminal 3ª VARA FEDERAL CRIMINAL DO RIO DE JANEIRO
Federal TRF2 Criminal 4ª VARA FEDERAL CRIMINAL DO RIO DE JANEIRO
Federal TRF2 Criminal 5ª VARA FEDERAL CRIMINAL DO RIO DE JANEIRO
Federal TRF2 Criminal 6ª VARA FEDERAL CRIMINAL DO RIO DE JANEIRO
Federal TRF2 Criminal 7ª VARA FEDERAL CRIMINAL DO RIO DE JANEIRO
Federal TRF2 Criminal 8ª VARA FEDERAL CRIMINAL DO RIO DE JANEIRO
Federal TRF2 Criminal 2ª VARA FEDERAL DE NITERÓI
Federal TRF2 Criminal 3ª VARA FEDERAL DE SÃO JOÃO DE MERITI
Federal TRF2 Criminal 1ª VARA FEDERAL CRIMINAL DE VITÓRIA
Federal TRF2 Criminal 2ª VARA FEDERAL CRIMINAL DE VITÓRIA
Federal TRF2 Criminal 10ª VARA FEDERAL CRIMINAL DO RIO DE JANEIRO
Federal TRF2 Criminal 6ª VARA FEDERAL DE SÃO JOÃO DE MERITI
Federal TRF2 Cível
Federal TRF2 Cível 15ª 

Federal TRF2 Juizados Cíveis 1JUIZADO ESPECIAL FEDERAL DE CACHOEIRO DE ITAPEMIRIM
Federal TRF2 Juizados Cíveis 2JUIZADO ESPECIAL FEDERAL DE CAMPOS
Federal TRF2 Juizados Cíveis 11º JUIZADO ESPECIAL FEDERAL
Federal TRF2 Juizados Cíveis 3º JUIZADO ESPECIAL FEDERAL DE SÃO GONÇALO
Federal TRF2 Juizados Cíveis 12º JUIZADO ESPECIAL FEDERAL DO RIO DE JANEIRO
Federal TRF2 Juizados Cíveis 13º JUIZADO ESPECIAL FEDERAL DO RIO DE JANEIRO
Federal TRF2 Juizados Cíveis 14º JUIZADO ESPECIAL FEDERAL DO RIO DE JANEIRO
Federal TRF2 Juizados Cíveis 15º JUIZADO ESPECIAL FEDERAL DO RIO DE JANEIRO
Federal TRF2 Juizados Cíveis 16º JUIZADO ESPECIAL FEDERAL DO RIO DE JANEIRO
Federal TRF3 Criminal
Federal TRF3 Criminal 1º VARA FEDERAL CRIMINAL DE SÃO PAULO
Federal TRF3 Criminal 2º VARA FEDERAL CRIMINAL DE SÃO PAULO
Federal TRF3 Criminal 3º VARA FEDERAL CRIMINAL DE SÃO PAULO
Federal TRF3 Criminal 4º VARA FEDERAL CRIMINAL DE SÃO PAULO
Federal TRF3 Criminal 5º VARA FEDERAL CRIMINAL DE SÃO PAULO
Federal TRF3 Criminal

Federal TRF3 Execução Fiscal 9º VARA FEDERAL DE RIBEIRÃO PRETO
Federal TRF3 Execução Fiscal 4º VARA FEDERAL DE SÃO JOSÉ DOS CAMPOS
Federal TRF3 Execução Fiscal 5º VARA FEDERAL DE SÃO JOSÉ DO RIO PRETO
Federal TRF3 Execução Fiscal 3º VARA FEDERAL DE GUARULHOS
Federal TRF3 Execução Fiscal 6º VARA FEDERAL DE CAMPO GRANDE
Federal TRF3 Execução Fiscal 2º VARA FEDERAL DE MOGI DAS CRUZES
Federal TRF3 Execução Fiscal 4º VARA FEDERAL DE SOROCABA
Federal TRF3 Execução Fiscal 5º VARA FEDERAL DE CAMPINAS
Federal TRF3 Execução Fiscal 7º VARA FEDERAL DE SANTOS
Federal TRF3 Execução Fiscal 1º VARA FEDERAL DE ANDRADINA
Federal TRF3 Execução Fiscal 1º VARA FEDERAL DE SÃO VICENTE
Federal TRF3 Execução Fiscal 13º VARA FEDERAL DE EXECUÇÕES FISCAIS DE SÃO PAULO
Federal TRF3 Geral
Federal TRF3 Geral 2º VARA FEDERAL DE RIBEIRÃO PRETO
Federal TRF3 Geral 4º VARA FEDERAL DE RIBEIRÃO PRETO
Federal TRF3 Geral 5º VARA FEDERAL DE RIBEIRÃO PRETO
Federal TRF3 Geral 6º VARA FEDERAL DE RIBEIRÃO PRETO
Federal TRF3 Geral

Federal TRF4 Criminal 11ª VARA FEDERAL DE PORTO ALEGRE
Federal TRF4 Criminal 1ª VARA FEDERAL DE CRICIÚMA
Federal TRF4 Criminal 7ª VARA FEDERAL DE FLORIANÓPOLIS
Federal TRF4 Criminal 1ª VARA FEDERAL DE ITAJAÍ
Federal TRF4 Criminal 1ª VARA FEDERAL DE JOINVILLE
Federal TRF4 Criminal 3ª VARA FEDERAL DE PASSO FUNDO
Federal TRF4 Criminal 1ª VARA FEDERAL DE FLORIANÓPOLIS
Federal TRF4 Criminal 9ª VARA FEDERAL DE CURITIBA
Federal TRF4 Criminal 1ª VARA FEDERAL DE CHAPECÓ
Federal TRF4 Criminal 5ª VARA FEDERAL DE CAXIAS DO SUL
Federal TRF4 Criminal 5ª VARA FEDERAL DE NOVO HAMBURGO
Federal TRF4 Criminal 1ª VARA FEDERAL DE GUAÍRA
Federal TRF4 Criminal 5ª VARA FEDERAL DE FOZ DO IGUAÇU
Federal TRF4 Criminal 3ª VARA FEDERAL DE MARINGÁ
Federal TRF4 Criminal 23ª VARA FEDERAL DE CURITIBA
Federal TRF4 Criminal GABINETE DO (A) DES. FEDERAL CLAUDIA CRISTINA CRISTOFANI
Federal TRF4 Criminal GABINETE DO (A) DES. FEDERAL JOÃO PEDRO GEBRAN NETO
Federal TRF4 Criminal GABINETE DO (A) DES. FEDERAL LEANDRO PAULSEN
F

Federal TRF4 Geral 1ª VARA FEDERAL DE CANOAS
Federal TRF4 Geral 2ª VARA FEDERAL DE CANOAS
Federal TRF4 Geral 1ª VARA FEDERAL DE CAXIAS DO SUL
Federal TRF4 Geral 2ª VARA FEDERAL DE CAXIAS DO SUL
Federal TRF4 Geral 2ª VARA FEDERAL DE ERECHIM
Federal TRF4 Geral 3ª VARA FEDERAL DE NOVO HAMBURGO
Federal TRF4 Geral 4ª VARA FEDERAL DE PASSO FUNDO
Federal TRF4 Geral 3ª VARA FEDERAL DE PELOTAS
Federal TRF4 Geral 25ª VARA FEDERAL DE PORTO ALEGRE
Federal TRF4 Geral 5ª VARA FEDERAL DE FLORIANÓPOLIS
Federal TRF4 Geral 4ª VARA FEDERAL DE ITAJAÍ
Federal TRF4 Geral 2ª VARA FEDERAL DE CAMPO MOURÃO
Federal TRF4 Geral CENTRO JUDICIÁRIO DE SOLUÇÃO DE CONFLITOS E CIDADANIA
Federal TRF4 Geral VICE-PRESIDÊNCIA
Federal TRF4 Geral 1VARA FEDERAL DE PITANGA
Federal TRF4 Geral 2ª VARA FEDERAL DE GRAVATAÍ
Federal TRF4 Geral 3ª VARA FEDERAL DE GRAVATAÍ
Federal TRF4 Juizados Cíveis


KeyboardInterrupt: 